In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import alexnet
from PIL import Image
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# -------------------------------------------------------------
# Load Pretrained AlexNet with 1024-D Output
# -------------------------------------------------------------
base_model = alexnet(weights='IMAGENET1K_V1')

# Freeze base model parameters (optional)
for param in base_model.parameters():
    param.requires_grad = False

# Custom model: extract features and add projection layer
alexnet_model = nn.Sequential(
    base_model.features,
    base_model.avgpool,
    nn.Flatten(),
    *list(base_model.classifier)[:5],  # Up to 4096-dim (before classifier[6])
    nn.Linear(4096, 1024),  # Project 4096 → 1024
    nn.ReLU()
)

alexnet_model.eval().to(device)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): AdaptiveAvgPool2d(output_size=(6, 6))
  (2): Flatten(start_dim=1, end_dim=-1)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=9216, out_features=4096, bias=True)
  (

In [3]:
# -------------------------------------------------------------
# Define Image Transformation
# -------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # AlexNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# -------------------------------------------------------------
# Feature Extraction Function
# -------------------------------------------------------------
def extract_features(image_path, transform, device=device):
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            features = alexnet_model(image).squeeze().cpu().numpy()

        return features
    except Exception as e:
        print(f"Skipping frame: {image_path} due to error: {e}")
        return None

In [12]:
# -------------------------------------------------------------
# Paths to RGB and Label CSV
# -------------------------------------------------------------
rgb_path = "D:/Datasets/Datasets/ADL/Frames/P_16"
label_csv_path = "D:/Datasets/Datasets/ADL/Label/P_16.csv"
output_csv = "Feature_AlexNet1024/Feature_P_16_ADL.csv"

labels_df = pd.read_csv(label_csv_path)

In [16]:
# -------------------------------------------------------------
# Extract Features and Save to CSV
# -------------------------------------------------------------
S = 40  # Sampling every 5th frame
features_list = []

all_frames = sorted(os.listdir(rgb_path))[::S]

for frame in tqdm(all_frames, desc="Extracting RGB Features"):
    rgb_frame_path = os.path.join(rgb_path, frame)
    rgb_features = extract_features(rgb_frame_path, transform, device)

    if rgb_features is not None:
        frame_number = int(frame.split('_')[-1].split('.')[0])
        label_row = labels_df[(labels_df['StartFrame'] <= frame_number) & (labels_df['EndFrame'] >= frame_number)]
        action_label = label_row.iloc[0]['ActionLabel'] if not label_row.empty else 0
        features_list.append([frame, action_label] + rgb_features.tolist())

# -------------------------------------------------------------
# Save Features to CSV
# -------------------------------------------------------------
if len(features_list) == 0:
    raise ValueError("No valid features extracted.")

columns = ["Frame", "ActionLabel"] + [f"Feature_{i}" for i in range(1024)]
df = pd.DataFrame(features_list, columns=columns)
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
df.to_csv(output_csv, index=False)

print(f"✅ Feature extraction using AlexNet completed! Saved to: {output_csv}")

Extracting RGB Features: 100%|███████████████████████████████████████████████████████| 630/630 [00:17<00:00, 36.23it/s]


✅ Feature extraction using AlexNet completed! Saved to: Feature_AlexNet1024/Feature_P_16_ADL.csv
